In [1]:

from datetime import datetime, timedelta
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from __future__ import division
import sys
!{sys.executable} -m pip install mlxtend
import mlxtend.preprocessing
import mlxtend.frequent_patterns
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

print(datetime.now())
myretaildata = pd.read_csv('data.csv')


2022-01-13 10:34:43.472007


In [16]:
myretaildata = myretaildata.loc[myretaildata['DATETIME__SALES_DATE']>'2021-12-01'].loc[myretaildata['INT_QUANTITY']>0].loc[:,['sales_no','item_name']] #datetime.strptime('20211201', '%Y%m%d').date()]


KeyError: 'DATETIME__SALES_DATE'

In [3]:
log = open('log3988.txt', 'w') 


In [4]:
myretaildata = myretaildata.groupby(['sales_no'])['item_name'].apply(','.join).reset_index()

In [5]:
print(myretaildata)

           sales_no                                          item_name
0              9001  #(B)AXCEL PARACETAMOL 250MG/5ML SYRUP (ORANGE)...
1              9002  #(B)AXCEL PARACETAMOL 250MG/5ML SYRUP (ORANGE)...
2              9003                             #(B)ISORADIN CREAM 10G
3              9004                          #(B)KLORAXIN EYE OINT. 5G
4              9005                          #(B)KLORAXIN EYE OINT. 5G
...             ...                                                ...
7146651  7530104012  #DICLAC RETARD 100MG 10S,COUNTERPAIN COOL 60G,...
7146652  7530104013  #H-CORT CR 15G,THE ORIGINAL VEGESORB CE MED+ M...
7146653  7530104014                                 #BILAXTEN 20MG 10S
7146654  7530104015  SUGO 365 VIP PACKAGE (RM360),CONTOUR PLUS TEST...
7146655  7530104016  CONTOUR PLUS ONE PLACEMENT METER (SUGO 365),CO...

[7146656 rows x 2 columns]


In [6]:
print(f'ready: {datetime.now()}')
invoice_item_list = []
for index,row in myretaildata.iterrows():
    tmp_items = list(row['item_name'].split(","))
    invoice_item_list.append(tmp_items)
    log.write(f"{index}:")    
    log.flush()
print(f'done: {datetime.now()}')
print(invoice_item_list)


ready: 2022-01-13 10:39:20.546717


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
# Initialize and fit the transaction encoder
online_encoder = mlxtend.preprocessing.TransactionEncoder()
online_encoder_array = online_encoder.fit(invoice_item_list)

print(len(invoice_item_list))

7146656


In [8]:
#abc = online_encoder.transform(invoice_item_list[:3])
#ab1c = online_encoder.transform(invoice_item_list[3:5])
#print(abc.shape)
#print(ab1c.shape)
#arr_ss =np.empty((3, len(online_encoder.columns_)), dtype=bool)
#print(arr_ss.shape)
#arr_ss = np.append(arr_s,abc, axis=0)
#arr_ss = np.append(abc,ab1c, axis=0)
#print(arr_ss)
#print(arr_ss)
#type(arr_ss)
#len(arr_ss)

#pd.DataFrame(arr_s, columns=online_encoder.columns_)
#etl_data_result = pd.concat(arr_s)

#df_final = pd.concat([df,df1])
#print(df_final)
#arr4=[]
#arr4.append(abc)
#arr4.append(ab1c)
#print(arr4[0])
#print(len(arr4[0][0]))
#print(len(online_encoder.columns_))
#

#print(arr4)
#df = pd.DataFrame(abc, columns=online_encoder.columns_)
#df1 = pd.DataFrame(ab1c, columns=online_encoder.columns_)

In [9]:
#online_encoder_array= online_encoder.transform(invoice_item_list)
def chunks_encode(l, n):
    """ Yield successive n-sized chunks from l.
    """
    arr_df = np.array([])
    df_final =pd.DataFrame()
    for i in range(0, len(l), n): # Use xrange if you're using Python 2 - it won't create the range list.
        log.write(f"{i}->{i+n}:")    
        log.flush()
        temp_arr = online_encoder.transform(l[i:i+n])
        if len(arr_df) ==0:
            arr_df = temp_arr
        else:
            np.append(arr_df,temp_arr,axis=0)
    df_final = pd.DataFrame(arr_df, columns=online_encoder.columns_)
    return df_final
online_encoder_df = chunks_encode(invoice_item_list,50000)


In [10]:
# Recast the encoded array as a dataframe
#online_encoder_df = pd.DataFrame(online_encoder_array, columns=online_encoder.columns_)

# Print the results
#online_encoder_df=online_encoder_array

In [11]:
apriori_model_colnames = mlxtend.frequent_patterns.apriori(
    online_encoder_df, 
    min_support=0.01,
    use_colnames=True
)

apriori_model_colnames

,support,itemsets
0,0.01058,(#ADEZIO 10MG 10X10S)
1,0.01516,(#ADEZIO ORAL SOLUTION 60ML)
2,0.01034,(#AVERA GEL SUGAR FREE 10G)
3,0.01296,(#BENA EXPECTORANT 120ML)
4,0.01002,(#CETYROL 10MG 10X10S)
...,...,...
206,0.03998,"(ALPRO MEMBERSHIP FREE RM10 ECOMMERCE VOUCHER,..."
207,0.03998,"(ALPRO LIFETIME VIP MEMBERSHIP, ALPRO MEMBERSH..."
208,0.07934,"(ALPRO MEMBERSHIP FREE RM10 ECOMMERCE VOUCHER,..."
209,0.03936,"(ALPRO MEMBERSHIP FREE RM10 ECOMMERCE VOUCHER,..."


In [12]:
apriori_model_colnames['length'] = (
    apriori_model_colnames['itemsets'].apply(lambda x: len(x))
)

apriori_model_colnames

,support,itemsets,length
0,0.01058,(#ADEZIO 10MG 10X10S),1
1,0.01516,(#ADEZIO ORAL SOLUTION 60ML),1
2,0.01034,(#AVERA GEL SUGAR FREE 10G),1
3,0.01296,(#BENA EXPECTORANT 120ML),1
4,0.01002,(#CETYROL 10MG 10X10S),1
...,...,...,...
206,0.03998,"(ALPRO MEMBERSHIP FREE RM10 ECOMMERCE VOUCHER,...",3
207,0.03998,"(ALPRO LIFETIME VIP MEMBERSHIP, ALPRO MEMBERSH...",3
208,0.07934,"(ALPRO MEMBERSHIP FREE RM10 ECOMMERCE VOUCHER,...",3
209,0.03936,"(ALPRO MEMBERSHIP FREE RM10 ECOMMERCE VOUCHER,...",4


In [13]:
rules = mlxtend.frequent_patterns.association_rules(
    apriori_model_colnames, 
    metric="lift",
    min_threshold=1.0, 
    support_only=False
)

rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(#BENA EXPECTORANT 120ML),(#ADEZIO ORAL SOLUTION 60ML),0.01296,0.01516,0.01008,0.777778,51.304603,0.009884,4.431780
1,(#ADEZIO ORAL SOLUTION 60ML),(#BENA EXPECTORANT 120ML),0.01516,0.01296,0.01008,0.664908,51.304603,0.009884,2.945576
2,(*ASTHALIN HFA INHALER 200DOSES),(#ADEZIO ORAL SOLUTION 60ML),0.01504,0.01516,0.01002,0.666223,43.946135,0.009792,2.950596
3,(#ADEZIO ORAL SOLUTION 60ML),(*ASTHALIN HFA INHALER 200DOSES),0.01516,0.01504,0.01002,0.660950,43.946135,0.009792,2.905057
4,(*VENTOLIN EVOHALER 100MCG 200DOSES),(#ADEZIO ORAL SOLUTION 60ML),0.01626,0.01516,0.01020,0.627306,41.379042,0.009953,2.642491
...,...,...,...,...,...,...,...,...,...
119,"(ALPRO MEMBERSHIP PACKAGE, ALPRO MEMBERSHIP FR...","(ALPRO MEMBERSHIP FREE RM10 ECOMMERCE VOUCHER,...",0.07934,0.03998,0.03998,0.503907,12.603983,0.036808,1.935162
120,(ALPRO MEMBERSHIP FREE RM10 ECOMMERCE VOUCHER),"(ALPRO LIFETIME VIP MEMBERSHIP, ALPRO MEMBERSH...",0.07934,0.03998,0.03998,0.503907,12.603983,0.036808,1.935162
121,(ALPRO LIFETIME VIP MEMBERSHIP),"(ALPRO MEMBERSHIP FREE RM10 ECOMMERCE VOUCHER,...",0.04160,0.07934,0.03998,0.961058,12.113155,0.036679,23.641640
122,(ALPRO MEMBERSHIP PACKAGE),"(ALPRO MEMBERSHIP FREE RM10 ECOMMERCE VOUCHER,...",0.07934,0.03998,0.03998,0.503907,12.603983,0.036808,1.935162


In [14]:
apriori_model_colnames.to_csv('3988_apriori.csv')
rules.to_csv('3988_asso_rules.csv')